In [5]:
%load_ext lab_black

from puzzles import load

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


**--- Day 7: Some Assembly Required ---**

This year, Santa brought little Bobby Tables a set of wires and bitwise logic gates! Unfortunately, little Bobby is a little under the recommended age range, and he needs help assembling the circuit.

Each wire has an identifier (some lowercase letters) and can carry a 16-bit signal (a number from 0 to 65535). A signal is provided to each wire by a gate, another wire, or some specific value. Each wire can only get a signal from one source, but can provide its signal to multiple destinations. A gate provides no signal until all of its inputs have a signal.

The included instructions booklet describes how to connect the parts together: x AND y -> z means to connect wires x and y to an AND gate, and then connect its output to wire z.

For example:

123 -> x means that the signal 123 is provided to wire x.
x AND y -> z means that the bitwise AND of wire x and wire y is provided to wire z.
p LSHIFT 2 -> q means that the value from wire p is left-shifted by 2 and then provided to wire q.
NOT e -> f means that the bitwise complement of the value from wire e is provided to wire f.
Other possible gates include OR (bitwise OR) and RSHIFT (right-shift). If, for some reason, you'd like to emulate the circuit instead, almost all programming languages (for example, C, JavaScript, or Python) provide operators for these gates.

For example, here is a simple circuit:

```
123 -> x
456 -> y
x AND y -> d
x OR y -> e
x LSHIFT 2 -> f
y RSHIFT 2 -> g
NOT x -> h
NOT y -> i
```

After it is run, these are the signals on the wires:

```
d: 72
e: 507
f: 492
g: 114
h: 65412
i: 65079
x: 123
y: 456
```

In little Bobby's kit's instructions booklet (provided as your puzzle input), what signal is ultimately provided to wire a?



In [15]:
len(sorted(load(7).strip().split("\n")))

339

In [37]:
to_do[:100]

[]

In [48]:
ss = sorted(load(7).strip().split("\n"))

In [49]:
[x for x in ss if x.endswith("-> a")]

['lx -> a']

In [53]:
[x for x in ss if x.endswith("-> lx")]

['lw OR lv -> lx']

In [54]:
[x for x in ss if "-> lw" in x]

['lc LSHIFT 1 -> lw']

In [55]:
[x for x in ss if "-> lv" in x]

['1 AND lu -> lv']

In [56]:
[x for x in ss if "-> lv" in x]

['1 AND lu -> lv']

In [ ]:
ss

In [36]:
funcs = {
    "AND": lambda a, b: (a & b) & 65535,
    "OR": lambda a, b: (a | b) & 65535,
    "NOT": lambda a: a ^ 65535,
    "RSHIFT": lambda a, b: (a >> b) & 65535,
    "LSHIFT": lambda a, b: (a << b) & 65535,
}

wires = {}

get_arg = lambda p: int(p) if p.isdecimal() else wires[p]

to_do = sorted(load(7).strip().split("\n"))

processed = []
errors = []

while len(to_do) > 0:
    n_processed = 0

    for line in to_do:
        left_part, out_name = line.split(" -> ")
        parts = left_part.split()

        try:
            # Assignment
            if len(parts) == 1:
                wires[out_name] = get_arg(parts[0])

            # Not
            elif len(parts) == 2:
                wires[out_name] = 65535 ^ wires[parts[-1]]

            # And, Or, Lshift, Rshift
            else:
                op = parts[1]
                arg_a = get_arg(parts[0])
                arg_b = get_arg(parts[2])
                wires[out_name] = 65535 & funcs[op](arg_a, arg_b)

            n_processed += 1
            processed.append(line)

        except Exception:
            errors.append(line)

    to_do = errors
    errors = []

wires["a"]

16076

---

**--- Part Two ---**

Now, take the signal you got on wire a, override wire b to that signal, and reset the other wires (including wire a). What new signal is ultimately provided to wire a?



In [32]:
wires["b"] = wires["a"]

for k in wires:
    if k == "b":
        continue
    wires[k] = 0

funcs = {
    "AND": lambda a, b: a & b,
    "OR": lambda a, b: a | b,
    "NOT": lambda a: a ^ 65535,
    "RSHIFT": lambda a, b: a >> b,
    "LSHIFT": lambda a, b: a << b,
}

wires = {"b": 16076}

get_arg = lambda p: int(p) if p.isdecimal() else wires[p]

to_do = sorted(load(7).strip().split("\n"))

processed = []
errors = []

wires_a = []

while len(to_do) > 0:
    n_processed = 0

    for line in to_do:
        # print(line)
        left_part, out_name = line.split(" -> ")
        parts = left_part.split()

        try:
            # Assignment
            if len(parts) == 1:
                wires[out_name] = get_arg(parts[0])

            # Not
            elif len(parts) == 2:
                wires[out_name] = 65535 ^ parts[-1]

            # And, Or, Lshift, Rshift
            else:
                op = parts[1]
                arg_a = get_arg(parts[0])
                arg_b = get_arg(parts[2])
                wires[out_name] = funcs[op](arg_a, arg_b)

            if wires["a"] is not None:
                wires_a.append(wires["a"])

            n_processed += 1
            processed.append(line)

        except Exception:
            errors.append(line)

    to_do = errors
    errors = []



KeyboardInterrupt: 

In [34]:
wires_a[-1]

IndexError: list index out of range

In [20]:
import matplotlib.pyplot as plt

----

In [57]:
ss[:3]

['0 -> c', '1 AND am -> an', '1 AND bh -> bi']

In [241]:
def is_number(t):
    return all([c in "1234567890" for c in t])


class Node:
    funcs = {
        "NOT": lambda a: a ^ 65535,
        "AND": lambda a, b: (a & b) & 65535,
        "OR": lambda a, b: (a | b) & 65535,
        "RSHIFT": lambda a, b: (a >> b) & 65535,
        "LSHIFT": lambda a, b: (a << b) & 65535,
    }

    @staticmethod
    def get_or_create(nodes, name):
        if name in nodes:
            node = nodes[name]
        else:
            node = Node(name)
            nodes[name] = node
        return node

    @staticmethod
    def build(line, nodes) -> "Node":
        what_to_do, name = line.split(" -> ")
        tokens = what_to_do.split()

        parent = Node.get_or_create(nodes, name)

        if len(tokens) == 1:
            # assign, like "123 -> x" or "a -> b"
            parent.type = "VALUE"
            if is_number(tokens[0]):
                parent.func = lambda: int(tokens[0])
            else:
                parent.func = lambda: None
                node = Node.get_or_create(nodes, tokens[0])
                node.parents.add(parent)
                parent.left = node

        elif len(tokens) == 2:
            # not, like "NOT x -> y"
            op_not, operand = tokens
            node = Node.get_or_create(nodes, operand)
            node.parents.add(parent)
            parent.type = op_not
            parent.left = node

        else:
            # and, or, lshift, rshift, like "ax AND by -> zq"
            left_part, op, right_part = tokens
            parent.type = op

            if is_number(left_part):
                left = Node(left_part, type="VALUE", func=lambda: int(left_part))
            else:
                left = Node.get_or_create(nodes, left_part)
            left.parents.add(parent)
            parent.left = left

            if is_number(right_part):
                right = Node(right_part, type="VALUE", func=lambda: int(right_part))
            else:
                right = Node.get_or_create(nodes, right_part)
            right.parents.add(parent)
            parent.right = right

    def __init__(self, name, type="VALUE", func=None):
        self.name = name
        self.type = type
        self.func = func

        self.value = None
        self.parents = set()
        self.left = None
        self.right = None

    def reset(self):
        self.value = None

    def process(self) -> int:
        print(f"[{self.name}]->", end="")
        if self.value is not None:
            return self.value

        if self.type == "VALUE":
            val = self.func()
            if val is None:
                # "y -> x"
                if self.left.value is None:
                    self.value = self.left.process()
                else:
                    self.value = self.left.value
                return self.value
            # "123 -> x"
            self.value = val
            return self.value

        f = Node.funcs[self.type]

        if self.left.value is None:
            left_value = self.left.process()

        if self.type == "NOT":
            self.value = f(left_value)
            return self.value

        if self.right.value is None:
            right_value = self.right.process()

        self.value = f(self.left.value, self.right.value)
        return self.value

    def __repr__(self):
        left = f"{self.left.name if self.left is not None else ''}"
        if self.type == "VALUE":
            return f"{self.func}[{self.value}]"
        if self.type == "NOT":
            return f"{self.type}[{left}]"
        right = f"{self.right.name if self.right is not None else ''}"
        return f"{self.type}[{left},{right}]"

In [242]:
lines = """123 -> x
x AND y -> d
456 -> y
NOT y -> i
NOT x -> h
x LSHIFT 2 -> f
y RSHIFT 2 -> g
x OR y -> e""".split(
    "\n"
)

In [249]:
lines = ss

In [250]:
nodes = {}
for i in range(len(lines)):
    Node.build(lines[i], nodes)

# nodes

In [254]:
for v in nodes.values():
    v.reset()
nodes["b"].value = 16076

In [255]:
nodes["a"].process()

[a]->[lx]->[lw]->[lc]->[lb]->[kh]->[kg]->[jm]->[jl]->[ir]->[iq]->[hw]->[hv]->[hb]->[ha]->[gg]->[gf]->[fl]->[fk]->[eq]->[ep]->[dv]->[du]->[da]->[cz]->[cf]->[ce]->[bk]->[bj]->[ap]->[ao]->[u]->[t]->[c]->[s]->[r]->[o]->[n]->[k]->[d]->[j]->[g]->[e]->[f]->[i]->[h]->[m]->[l]->[q]->[p]->[an]->[am]->[aj]->[x]->[v]->[w]->[ai]->[af]->[y]->[ae]->[ab]->[z]->[aa]->[ad]->[ac]->[ah]->[ag]->[al]->[ak]->[bi]->[bh]->[be]->[as]->[aq]->[ar]->[bd]->[ba]->[at]->[az]->[aw]->[au]->[av]->[ay]->[ax]->[bc]->[bb]->[bg]->[bf]->[cd]->[cc]->[bz]->[bn]->[bl]->[bm]->[by]->[bv]->[bo]->[bu]->[br]->[bp]->[bq]->[bt]->[bs]->[bx]->[bw]->[cb]->[ca]->[cy]->[cx]->[cu]->[ci]->[cg]->[ch]->[ct]->[cq]->[cj]->[cp]->[cm]->[ck]->[cl]->[co]->[cn]->[cs]->[cr]->[cw]->[cv]->[dt]->[ds]->[dp]->[dd]->[db]->[dc]->[do]->[dl]->[de]->[dk]->[dh]->[df]->[dg]->[dj]->[di]->[dn]->[dm]->[dr]->[dq]->[eo]->[en]->[ek]->[dy]->[dw]->[dx]->[ej]->[eg]->[dz]->[ef]->[ec]->[ea]->[eb]->[ee]->[ed]->[ei]->[eh]->[em]->[el]->[fj]->[fi]->[ff]->[et]->[er]->[es]->[fe]-

2797